In [17]:
import robin_stocks as r
from datetime import datetime
from dateutil import tz
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def diffMaxHighLow(df, strike_price,volume):
    print('Strike\tVolume\tHigh\tLow\tChange\tPercent')
    high_price = max(df.high_price)
    low_price = min(df.high_price)
    diff = round((high_price - low_price),2)
    percent = round(diff/low_price,2)
    print('${:.2f}\t{:,.0f}\t{}\t{}\t{}\t{:,.0%}'.format(strike_price,volume,high_price,low_price,diff,percent))

def figForExpriation(strike_price,volume,optionType,span):
    #df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
    df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span))
    df['strike_price'] = strike_price
    df[['strike_price','high_price','low_price']] = df[['strike_price','high_price','low_price']].apply(pd.to_numeric)
    df = df[df.high_price != df.high_price[0:1][0]]   #filter no changes form the starting date
    print()
    diffMaxHighLow(df,float(strike_price),float(volume))
    local = utcToLocal(df.iloc[-1:].begins_at.values[-1])
    title = '{} Expiration:{}, Time:{:{dfmt} {tfmt}}'.format(symbol,expirationDate,local,dfmt='%Y-%m-%d', tfmt='%H:%M')
    fig = px.line(df, x="begins_at", y="close_price", color="strike_price", line_group="strike_price", hover_name="strike_price",
            line_shape="spline", render_mode="svg")
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Option price",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        ))
    fig.show()

### Expiration Dates

In [2]:
symbol = 'CLSK'
expiration_dates = r.get_chains(symbol)['expiration_dates']
print(list(enumerate(expiration_dates)))

[(0, '2020-09-18'), (1, '2020-10-16'), (2, '2020-12-18'), (3, '2021-03-19')]


### Strike Prices for Expiration Date

In [3]:
expirationDate = expiration_dates[1]
volume_limit = 0
optionData = r.find_options_by_expiration([symbol], expirationDate=expirationDate,optionType='call') 
dfoptions = pd.DataFrame((filter(lambda x:x['volume']>volume_limit,optionData)))
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='strike_price')
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='volume',ascending=False)
# df1 = dfoptions[['strike_price','volume']][dfoptions.strike_price=='237.5000']
# dfstrike_prices = pd.concat([df1,dfstrike_prices])
dfstrike_prices = dfoptions.sort_values(by='volume',ascending=False)[0:5][['strike_price','volume']].apply(pd.to_numeric)
dfstrike_prices.sort_values(by='strike_price',ascending=True,inplace=True)
dfstrike_prices

,strike_price,volume
6,7.5,5
0,10.0,192
1,12.5,64
2,15.0,9
5,22.5,10


In [8]:
 dfstrike_prices.iloc[0].strike_price

7.5

In [16]:
pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span))

,begins_at,open_price,close_price,high_price,low_price,volume,session,interpolated,symbol
0,2020-09-08T00:00:00Z,0.010000,2.780000,3.150000,0.010000,0,reg,False,CLSK
1,2020-09-09T00:00:00Z,2.780000,2.780000,2.780000,2.780000,0,reg,True,CLSK
2,2020-09-10T00:00:00Z,2.780000,2.780000,2.780000,2.780000,0,reg,True,CLSK
3,2020-09-11T00:00:00Z,2.780000,2.780000,2.780000,2.780000,0,reg,True,CLSK


In [19]:
optionType = 'call'
span = 'day'
idx = 0
strike_price = dfstrike_prices.iloc[idx].strike_price
volume = dfstrike_prices.iloc[idx].volume
#strike_price = 40.0
figForExpriation(strike_price,volume,optionType,span)

5.0

In [20]:
optionType = 'call'
span = 'week'
figForExpriation(strike_price,volume,optionType,span)


400 Client Error: Bad Request for url: https://api.robinhood.com/marketdata/options/historicals/a5626a02-4768-42e9-8aad-ad523a4b16b7/?span=week&interval=week&bounds=regular


KeyError: "['high_price', 'low_price'] not in index"

In [60]:
span = 'year'
figForExpriation(strike_price,volume,optionType,span)

*....
Strike	Volume	High	Low	Change	Percent
$64.00	29	2.7	0.01	2.69	26,900%
